# Information Retrieval Lab WiSe 2024/2025: Baseline Retrieval System

This Jupyter notebook serves as a baseline retrieval system that you can improve upon.
We use subsets of the MS MARCO datasets to retrieve passages of web documents.
We will show you how to create a software submission to TIRA from this notebook.

An overview of all corpora that we use in the current course is available at [https://tira.io/datasets?query=ir-lab-wise-2024](https://tira.io/datasets?query=ir-lab-wise-2024). The dataset IDs for loading the datasets are:

- `ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training`: A subsample of the TREC 2019/2020 Deep Learning tracks on the MS MARCO v1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/subsampled-ms-marco-rag-20241202-training` (_work in progress_): A subsample of the TREC 2024 Retrieval-Augmented Generation track on the MS MARCO v2.1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/ms-marco-rag-20241203-test` (work in progress): The test corpus that we have created together in the course, based on the MS MARCO v2.1 passage dataset. We will use this dataset as the test dataset, i.e., evaluation scores become available only after the submission deadline.

### Step 1: Import libraries

We will use [tira](https://tira.io/), an information retrieval shared task platform, and [ir_dataset](https://ir-datasets.com/) for loading the datasets. Subsequently, we will build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine framework.

First, we need to install the required libraries.

In [1]:
!pip3 install 'tira>=0.0.139' ir-datasets 'python-terrier==0.10.0'

Create an API client to interact with the TIRA platform (e.g., to load datasets and submit runs).

In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



### Step 2: Load the dataset

We load the dataset by its ir_datasets ID (as listed in the Readme). Just be sure to add the `irds:` prefix before the dataset ID to tell PyTerrier to load the data from ir_datasets.

In [3]:
from pyterrier import get_dataset

pt_dataset = get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')

### Step 3: Build an index

We will then create an index from the documents in the dataset we just loaded.

In [4]:
from pyterrier import IterDictIndexer

indexer = IterDictIndexer(
    # Store the index in the `index` directory.
    "../data/index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)
index = indexer.index(pt_dataset.get_corpus_iter())

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:   0%|          | 0/68261 [00:0…

00:22:01.223 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6114613) - further warnings are suppressed
00:22:07.627 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents


### Step 4: Define the retrieval pipeline

We will define a simple retrieval pipeline using just BM25 as a baseline. For details, refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io) or [tutorial](https://github.com/terrier-org/ecir2021tutorial).

In [5]:
from pyterrier import BatchRetrieve

bm25 = BatchRetrieve(index, wmodel="BM25")

### Step 5: Create the run
In the next steps, we would like to apply our retrieval system to some topics, to prepare a 'run' file, containing the retrieved documents.

First, let's have a short look at the first three topics:

In [6]:
# The `'text'` argument below selects the topics `text` field as the query.
df = pt_dataset.get_topics('text')
pt_dataset.get_topics('text')

,qid,query
0,1030303,who is aziz hashim
1,1037496,who is rep scalise
2,1043135,who killed nicholas ii of russia
3,1051399,who sings monk theme song
4,1064670,why do hunters pattern their shotguns
...,...,...
92,405717,is cdg airport in main paris
93,182539,example of monotonic function
94,1113437,what is physical description of spruce
95,1129237,hydrogen is a liquid below what temperature


Now, retrieve results for all the topics (may take a while):

In [7]:
run = bm25(pt_dataset.get_topics('text'))

That's it for the retrieval. Here are the first 10 entries of the run:

In [8]:
run.head(10)

,qid,docid,docno,rank,score,query
0,1030303,53852,8726436,0,31.681671,who is aziz hashim
1,1030303,56041,8726433,1,25.966276,who is aziz hashim
2,1030303,62116,8726435,2,23.863442,who is aziz hashim
3,1030303,32183,8726429,3,23.391821,who is aziz hashim
4,1030303,35867,8726437,4,21.030669,who is aziz hashim
5,1030303,17637,8726430,5,19.967200,who is aziz hashim
6,1030303,42957,7156982,6,19.967200,who is aziz hashim
7,1030303,21803,8726434,7,19.474804,who is aziz hashim
8,1030303,59828,1305520,8,17.849161,who is aziz hashim
9,1030303,60002,3302257,9,17.832781,who is aziz hashim


### Step 6: Evaluate your run

In [9]:
from pyterrier import Experiment

Experiment([bm25],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    eval_metrics = ["map", "recip_rank", "ndcg_cut_10", "P_1", "P_5", "P_10"]
)

,name,map,recip_rank,ndcg_cut_10,P_1,P_5,P_10
0,BR(BM25),0.412718,0.786653,0.489469,0.701031,0.62268,0.574227


### Step 7: Improve

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [16]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# 1. Text Cleaning/ Preprocessing & Re-indexing

* Removes URLs and special characters.
* Lowercases everything.
* Tokenizes and removes stopwords.
* Returns cleaned text.
* Reads the original corpus, cleans it, and creates a second index (../data/cleaned_index) specifically for the cleaned documents.


In [17]:
import pyterrier as pt
import re
import nltk

# Ensure you have the required nltk resources
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Preprocess text to remove URLs, clean text, and remove stop words
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)

    # Remove special characters and lowercasing
    text = re.sub(r'[^\w\s]', '', text.lower())

    # Tokenize text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

# Clean corpus
def clean_corpus(corpus_iter):
    for doc in corpus_iter:
        docno = doc['docno']
        text = doc['text']
        cleaned_text = preprocess_text(text)
        yield {'docno': docno, 'text': cleaned_text}

# Apply cleaning to the corpus
cleaned_corpus_iter = clean_corpus(pt_dataset.get_corpus_iter())

# Create a new index with cleaned corpus
indexer = IterDictIndexer(
    "../data/cleaned_index",
    meta={'docno': 50, 'text': 4096},
    overwrite=True,
)

index = indexer.index(cleaned_corpus_iter)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:   0%|          | 0/68261 [00:0…

00:42:23.777 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (976964) - further warnings are suppressed
00:42:39.976 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 4 empty documents


In [24]:
!pip install transformers torch


# 2. Sentence-BERT for Embeddings

* Loads a pre-trained MiniLM model for sentence embeddings

In [30]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example query
query = "who is aziz hashim"

# Generate an embedding for the query
query_embedding = model.encode(query)

# You can then use a nearest-neighbors search for similar queries or documents
# This is a more direct approach for semantic search rather than generation


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 3. BM25 Retrieval on Cleaned Index

In [32]:
from pyterrier import BatchRetrieve

# Assuming index is already built
bm25 = BatchRetrieve(index, wmodel="BM25")

# Get original queries (or the expanded queries if you want to test them)
queries = pt_dataset.get_topics('text')

# Run BM25 on original queries
bm25_results = bm25(queries)


In [34]:
print(queries.columns)  # Check the columns in the queries DataFrame


Index(['qid', 'query'], dtype='object')


In [37]:
topics = pt_dataset.get_topics('text')
print(topics)


        qid                                              query
0   1030303                                 who is aziz hashim
1   1037496                                 who is rep scalise
2   1043135                   who killed nicholas ii of russia
3   1051399                          who sings monk theme song
4   1064670              why do hunters pattern their shotguns
..      ...                                                ...
92   405717                       is cdg airport in main paris
93   182539                      example of monotonic function
94  1113437             what is physical description of spruce
95  1129237        hydrogen is a liquid below what temperature
96   146187  difference between a mcdouble and a double che...

[97 rows x 2 columns]


In [39]:
import torch

# 4. Semantic Search (Sentence-BERT)

* Converts all documents in the corpus to embeddings.
* Converts the queries to embeddings.
* Uses cosine similarity to find top-k similar document


In [40]:
from sentence_transformers import SentenceTransformer, util
from pyterrier import BatchRetrieve

# Load the pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Assuming pt_dataset.get_topics('query') gives a DataFrame with columns ['qid', 'query']
queries = pt_dataset.get_topics('query')

# Generate embeddings for queries
query_embeddings = model.encode(queries['query'], convert_to_tensor=True)

# Assuming BM25 is already set up
bm25 = BatchRetrieve(index, wmodel="BM25")

# Perform retrieval using BM25 (baseline)
bm25_results = bm25(queries)

# For semantic search with Sentence-BERT, compare cosine similarities
documents = [doc['text'] for doc in pt_dataset.get_corpus_iter()]
document_embeddings = model.encode(documents, convert_to_tensor=True)

# Compare the query embeddings with document embeddings using cosine similarity
cosine_scores = util.pytorch_cos_sim(query_embeddings, document_embeddings)

# You can retrieve the top K most relevant documents for each query
top_k_results = []
top_k = 10

for i, scores in enumerate(cosine_scores):
    sorted_indices = torch.argsort(scores, descending=True)[:top_k]
    top_k_results.append([documents[idx] for idx in sorted_indices])




ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:   0%|          | 0/68261 [00:0…

# 5. Wrapper Class for Semantic Search

A custom PyTerrier Transformer that:
* Takes precomputed document embeddings and a Sentence-BERT model.
* For each query in a dataframe, encodes it, does cosine similarity, and picks top-k docs.
* Returns a DataFrame of results with columns ['qid', 'docno', 'score'].


In [62]:
from pyterrier import Transformer
from sentence_transformers import SentenceTransformer, util
import pandas as pd

class SemanticSearchWrapper(Transformer):
    def __init__(self, query_embeddings, document_embeddings, k=10):
        self.query_embeddings = query_embeddings
        self.document_embeddings = document_embeddings
        self.k = k
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

    def transform(self, topics):
        all_documents = [doc['text'] for doc in pt_dataset.get_corpus_iter()]
        all_docnos = [doc['docno'] for doc in pt_dataset.get_corpus_iter()]  # Get all docnos

        results_list = []
        for i, query in enumerate(topics['query']):
            query_embedding = self.model.encode(query, convert_to_tensor=True)
            cosine_scores = util.pytorch_cos_sim(query_embedding, self.document_embeddings)

            # Get top k indices and ensure the length is at least k
            sorted_indices = torch.argsort(cosine_scores, descending=True)[:self.k].flatten().tolist()
            num_results = len(sorted_indices) # get actual number of results

            # Create a DataFrame for this query's results, ensuring all columns have the same length
            df = pd.DataFrame({
                'qid': [topics['qid'][i]] * num_results,  # Repeat qid for each result
                'docno': [all_docnos[idx] for idx in sorted_indices],  # Use corresponding docnos
                'score': cosine_scores[0][sorted_indices].cpu().tolist()  # Add cosine similarity scores
            })
            results_list.append(df)

        # Concatenate all query results into a single DataFrame
        final_results_df = pd.concat(results_list, ignore_index=True)
        return final_results_df  # Return a DataFrame

In [63]:
from pyterrier import Experiment

# Create an instance of the SemanticSearchWrapper
semantic_search_instance = SemanticSearchWrapper(query_embeddings, document_embeddings)

# Perform evaluation for both BM25 and semantic search pipelines
results = Experiment([bm25, semantic_search_instance],  # bm25 and semantic_search_instance are now two retrieval pipelines
                     queries,                        # List of queries or topics
                     pt_dataset.get_qrels(),         # Ground truth relevance data (qrels)
                     eval_metrics=["map", "ndcg_cut_10", "P_1", "P_5", "P_10"])

# Print the results to compare BM25 and semantic search performance
print(results)

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:   0%|          | 0/68261 [00:0…

ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training documents:   0%|          | 0/68261 [00:0…

                                                name       map  ndcg_cut_10  \
0                                           BR(BM25)  0.408130     0.489501   
1  <__main__.SemanticSearchWrapper object at 0x78...  0.531523     0.652962   

        P_1       P_5      P_10  
0  0.690722  0.614433  0.570103  
1  0.876289  0.793814  0.737113  


In [26]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn"  # Facebook's BART model
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Function to expand the query using the BART model
def expand_query_with_bart(query):
    input_text = f"expand: {query}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the expanded query
    outputs = model.generate(inputs["input_ids"], max_length=60, num_beams=5, early_stopping=True)

    # Decode the generated text
    expanded_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return expanded_query

# Example usage
expanded_query = expand_query_with_bart("who is aziz hashim")
print(expanded_query)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

expand: who is aziz hashim? Find out more at azizhashim.com. For confidential support call the Samaritans in the UK on 08457 90 90 90 or visit a local Samaritans branch, see www.samaritans.org for details. In
